# Read Mission Data

Reads mission data from the files cluster_data.csv and gaia_data.csv and transforms these into RDF data.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pprint import pprint
import numpy as np
import pandas as pd
import time
from tqdm.notebook import trange, tqdm
import pdb

In [3]:
from convert_csv import row2rdf, renameDF, strOrNum

In [4]:
prefixes = ["""@base <http://ioa-graph/resource/> .
@prefix ioa: <http://www.semanticweb.org/esa-ioa/ontologies/2022/ioa-wiki-ontology#> .
@prefix pred: <http://www.semanticweb.org/esa-ioa/ontologies/2022/predicates-ontology#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> ."""]

## GAIA

In [5]:
gaia_df = pd.read_csv('gaia_data.csv')
gaia_df

,class,name,label,known_as,parent,attributes
0,Mission,gaia,GAIA,gaia;gaia mission,NaN,NaN
1,SpaceSegment,ss,Space Segment,space segment,gaia,NaN
2,GroundSegment,gs,Ground Segment,ground segment,gaia,NaN
3,Spacecraft,sc,Spacecraft,gaia;spacecraft;2013-074A;39479,ss,DiscosID:39476
4,Subsystem,svm,Service Module (SVM),service module,sc,NaN
5,Subsystem,esvm,Electrical Service Module (E-SVM),electrical service module,svm,NaN
6,Subsystem,msvm,Mechanical Service Module (M-SVM),mechanical service module,svm,NaN
7,Subsystem,cdms,Control and Data Management System (CDMS),obdh;data management system,svm,NaN
8,Subsystem,cdmu,Control and Data Management Unit (CDMU),obdh;data management unit,cdms,NaN
9,Subsystem,aocs,Attitude and Orbit Control System (AOCS),attitude control system;orbit control system,svm;esvm;msvm,NaN


In [6]:
df_transform = renameDF(gaia_df)
df_transform.head()

,class,name,label,known_as,parent,attributes
0,Mission,gaia_mission,GAIA,gaia;gaia mission,NaN,NaN
1,SpaceSegment,gaia_ss,GAIA Space Segment,space segment,[gaia_mission],NaN
2,GroundSegment,gaia_gs,GAIA Ground Segment,ground segment,[gaia_mission],NaN
3,Spacecraft,gaia_sc,GAIA Spacecraft,gaia;spacecraft;2013-074A;39479,[gaia_ss],DiscosID:39476
4,Subsystem,gaia_svm,GAIA Service Module (SVM),service module,[gaia_sc],NaN


In [7]:
df_transform.tail()

,class,name,label,known_as,parent,attributes
21,Subsystem,gaia_ps,GAIA Propulsion System (PS),ps;propulsion system;reaction control system;rcs,"[gaia_aocs, gaia_msvm]",NaN
22,Subsystem,gaia_cps,GAIA Chemical Propulsion System (CPS),cps;chemical propulsion system,[gaia_ps],NaN
23,Subsystem,gaia_cpsa,GAIA Chemical Propulsion System A (CPS-A),cpsa;chemical propulsion system a;cps-a,[gaia_cps],NaN
24,Subsystem,gaia_cpsb,GAIA Chemical Propulsion System B (CPS-B),cpsb;chemical propulsion system b;cps-b,[gaia_cps],NaN
25,Subsystem,gaia_mps,GAIA Micro-Propulsion System (MPS),mps;micro propulsion system,[gaia_ps],NaN


In [8]:
row_test = df_transform.loc[3]
rdf_str_test = row2rdf(row_test)
pprint(rdf_str_test)

('<gaia_sc> a ioa:Spacecraft ;\n'
 '  pred:instance_of ioa:Spacecraft ;\n'
 "  pred:name 'gaia'^^xsd:string ;\n"
 "  pred:name 'spacecraft'^^xsd:string ;\n"
 "  pred:name '2013-074A'^^xsd:string ;\n"
 "  pred:name '39479'^^xsd:string ;\n"
 '  ioa:parent <gaia_ss> ;\n'
 "  rdfs:label 'GAIA Spacecraft'^^xsd:string .\n"
 '\n'
 '<gaia_sc> ioa:parent <obj39476> .\n'
 '\n'
 '<obj39476> a ioa:Spacecraft ;\n'
 '  pred:instance_of ioa:Spacecraft ;\n'
 '  ioa:parent <gaia_ss> .')


In [9]:
gaia_rdf = prefixes.copy()

for index, row in df_transform.iterrows():
    gaia_rdf.append(row2rdf(row))

with open('gaia_rdf.ttl', 'w') as f:
    f.write('\n\n'.join(gaia_rdf))

## Cluster

In [10]:
cluster_df = pd.read_csv('cluster_data.csv')
cluster_df

,class,name,label,known_as,parent,attributes
0,Mission,cluster,Cluster II,cluster;cluster ii;cluster mission,NaN,NaN
1,SpaceSegment,ss,Space Segment,space segment;cluster space segment,cluster,NaN
2,GroundSegment,gs,Ground Segment,ground segment;cluster ground segment,cluster,NaN
3,Constellation,constellation,Satellite Constellation,constellation;cluster constellation;cluster ii,ss,NaN
4,Spacecraft,clu1,Salsa,clu1;salsa;cluster ii fm6;fm6;2000-041b;26411,constellation,DiscosID:26408
...,...,...,...,...,...,...
191,Subsystem,clu4_wbd,Wide Band Data (WBD),wbd;wide band data,clu4_wec,NaN
192,Subsystem,clu4_dwp,Digital Wave Processor (DWP),dwp;digital wave processor,clu4_wec,NaN
193,Subsystem,clu4_pwr,Power Supply (PWR),pwr;power supply,clu4_wec,NaN
194,Subsystem,clu4_staff,Spatio-Temporal Analysis of Field Fluctuations...,staff;spatio-temporal analysis of field fluctu...,clu4_wec,NaN


In [11]:
df_transform_cluster = renameDF(cluster_df.copy())
df_transform_cluster.tail(10)

,class,name,label,known_as,parent,attributes
186,Subsystem,cluster_clu4_leea,Cluster II Low Energy Electron Analyser (LEEA),leea;low energy electron analyser,[cluster_clu4_peace],NaN
187,Subsystem,cluster_clu4_dpu,Cluster II Data Processing Unit (DPU),dpu;data processing unit,[cluster_clu4_peace],NaN
188,Subsystem,cluster_clu4_rapid,Cluster II Research with Adaptive Particle Ima...,rapid;research with adaptive particle imaging ...,[cluster_clu4_payload],NaN
189,Subsystem,cluster_clu4_wec,Cluster II Wave Experiment Consortium (WEC),wec;wave experiment consortium,[cluster_clu4_payload],NaN
190,Subsystem,cluster_clu4_efw,Cluster II Electric Field and Wave Experiment ...,efw;electric field and wave experiment,[cluster_clu4_wec],NaN
191,Subsystem,cluster_clu4_wbd,Cluster II Wide Band Data (WBD),wbd;wide band data,[cluster_clu4_wec],NaN
192,Subsystem,cluster_clu4_dwp,Cluster II Digital Wave Processor (DWP),dwp;digital wave processor,[cluster_clu4_wec],NaN
193,Subsystem,cluster_clu4_pwr,Cluster II Power Supply (PWR),pwr;power supply,[cluster_clu4_wec],NaN
194,Subsystem,cluster_clu4_staff,Cluster II Spatio-Temporal Analysis of Field F...,staff;spatio-temporal analysis of field fluctu...,[cluster_clu4_wec],NaN
195,Subsystem,cluster_clu4_whisper,Cluster II Waves of HIgh frequency and Sounder...,whisper;waves of high frequency and sounder fo...,[cluster_clu4_wec],NaN


In [12]:
cluster_rdf = prefixes.copy()

for index, row in df_transform_cluster.iterrows():
    cluster_rdf.append(row2rdf(row))

with open('cluster_rdf.ttl', 'w') as f:
    f.write('\n\n'.join(cluster_rdf))